## About the Data

The EMIT instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with JPL Open Science and Open Data policy.

## Requirements

- NASA Earthdata Account
- Set up Python Environment - See setup_instructions.md in the '/setup/' folder

## Learning Objectives

- How to use 'U.S.GHG Center STAC Catalog' to access 'EMIT Methane Point Source Plume Complexes' data.
- How to visualize ISS Spatial coverage and detected plumes target mask.
- How to visualize datasets using 'folium'.

## Approach

1. Visualize obsereved masks of the ISS and target masks of detected plumes.  
2. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Earth Surface Mineral Dust Source Investigation (EMIT) methane emission plumes data product.
3. Pass the STAC item into the raster API `/stac/tilejson.json` endpoint.
4. Using `folium.Map`, visualize the plumes.

   

# Setup
Import the required Python libraries.

In [ ]:
# Import required libraries

# import earthaccess
import warnings
import requests
import pandas as pd
import geopandas
import folium
import folium.plugins
import numpy as np
import matplotlib.pyplot as plt
import branca.colormap as cm
import seaborn as sns

from folium import Map, TileLayer 
from branca.element import Figure
from pystac_client import Client 
from pyproj import Geod
from shapely import wkt

## NASA Earth Data Login Credentials

To download or stream NASA data you will need an Earthdata account, you can create one here https://urs.earthdata.nasa.gov/home. We will use the 'login' function from the 'earthaccess' library for authentication before downloading at the end of the notebook. This function can also be used to create a local '.netrc' file if it doesn’t exist or add your login info to an existing '.netrc' file. If no Earthdata Login credentials are found in the '.netrc' you’ll be prompted for them. This step is not necessary to conduct searches but is needed to download or stream data.

## Define Spatial Region of Interest

For this example, our spatial region of interest (ROI) will be the Permian Basin, USA.

In [ ]:
# Loading Permian Basin shape file as a region of interest
# User can pass any json or shape file here
permian_basin = geopandas.read_file('permian.zip')

In [ ]:
# ISS data coverage 
coverage = geopandas.read_file('coverage.json')
coverage['new_start_time'] = coverage.start_time.astype('str')

# location where plume was detected
metadata_json = geopandas.read_file('methane_metadata.json')
metadata_json['new_start_time'] = metadata_json['UTC Time Observed'].astype('str')
metadata_json['Scene FIDs'] = list(map(lambda x : x[0] ,metadata_json['Scene FIDs']))

target_mask = geopandas.read_file('Target_Mask.json')

# ISS area intersecting with permian basin
result_coverage = geopandas.clip(coverage, permian_basin)
# result_metadata = geopandas.clip(metadata_json, permian_basin)

## Opening and Exploring Spatial Coverage of ISS and Mask of Identified Plumes

In this cell, we will visualize the outputs on a map using 'folium'.

In [ ]:
# Initializing a map with a center at [43, -100] and a zoom level of 4. The use of tiles = None has been employed to remove the default basemap.

colormap = cm.linear.YlGn_09.scale(
    coverage.start_time.min(), coverage.start_time.max()
)
m_ = folium.Map(location=[43, -100], zoom_start=4, tiles=None)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(m_)
# Load other TileLayer from https://leaflet-extras.github.io/leaflet-providers/preview/

# From covergae.json fid, geometry has been selected
map_layer_coverage_target = folium.GeoJson(target_mask, name = ' Target Mask')
map_layer_coverage_metadata = folium.GeoJson(metadata_json[['Scene FIDs', 'geometry', 'new_start_time']], name = ' Metadata')


map_layer_coverage_target.add_to(m_)
# map_layer_coverage_target.marker(icon=folium.Icon(color='blue', icon='star'))
map_layer_coverage_metadata.add_to(m_)
coverage[['fid', 'geometry', 'new_start_time']].explore(
    "fid",
    categorical=False,
    tooltip=[
        "fid",
        "new_start_time",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="Coverage",
    m=m_,
    legend=False,
    show=False
)


# Layer control crates the toggle button for all the layers
folium.LayerControl(collapsed=False).add_to(m_)
m_

## Opening and Exploring EMIT Methane Point Source Plume Complexes Data Using the Raster API
Search for EMIT Methane Point Source Plume Complexes Data

In [ ]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://ghg.center/api/stac"
RASTER_API_URL = "https://ghg.center/api/raster"

# Please use the collection name similar to the one used in STAC collection.
# Name of the collection for methane emission plumes. 
collection_name = "emit-ch4plume-v1"

In [ ]:
# Fetching the collection from STAC collections using appropriate endpoint.
# the 'requests' library allows a HTTP request possible
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection = {key: value for key, value in collection.items() if key != 'summaries'}
collection


Examining the contents of our `collection` under the `temporal` variable, we note that data is available from August 2022 to May 2023. By looking at the `dashboard: time density`, we can see that observations are conducted daily and non-periodically (i.e., there are plumes emissions for multiple places on the same dates).

In [ ]:
# Create a function that would search for the number of items in above data collection in the STAC API
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [ ]:
# Apply the above function and check the total number of items available within the collection
number_of_items = get_item_count(collection_name)
plume_complexes = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
parse_plume_complexes = plume_complexes
print(f"Found {len(plume_complexes)} items")

In [ ]:
# Examine the first item in the collection
plume_complex_ids = list(map(lambda d: d.get('id', f"id not found in dictionary"), plume_complexes))
plume_complex_ids[:10]

In [ ]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
plume_complexes = {plume_complex["id"]: plume_complex for plume_complex in plume_complexes} 
asset_name = "ch4-plume-emissions"

Below, we are entering the minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

In [ ]:
# Fetching the min and max values for a specific item
rescale_values = {"max":plume_complexes[list(plume_complexes.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":plume_complexes[list(plume_complexes.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this for only one item so that we can visualize the event. 

In [ ]:
color_map = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.
plume_complex_id_1 = "EMIT_L2B_CH4PLM_001_20230418T200118_000829"
methane_plume_tile_1 = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={plume_complexes[plume_complex_id_1]['collection']}&item={plume_complexes[plume_complex_id_1]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
methane_plume_tile_1

In [ ]:
color_map = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.
plume_complex_id_2 = "EMIT_L2B_CH4PLM_001_20230614T193706_000038"
methane_plume_tile_2 = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={plume_complexes[plume_complex_id_2]['collection']}&item={plume_complexes[plume_complex_id_2]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
methane_plume_tile_2

## Visualizing CH₄ Emission Plume


In [ ]:
colormap = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

#Defining the breaks in the colormap 
color_map = cm.LinearColormap(colors = ['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'], vmin = 0, vmax = 1500 )
color_map.caption = 'ppm-m'

# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.

# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(methane_plume_tile_1["center"][1], methane_plume_tile_1["center"][0]), zoom_start=13, tiles=None, tooltip = 'test tool tip')
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)


map_layer = TileLayer(
    tiles=methane_plume_tile_1["tiles"][0],
    name='Plume Complex Landfill',
    overlay='True',
    attr="GHG",
    opacity=1,
)
map_layer.add_to(map_)
folium.LayerControl(collapsed=False, position='bottomleft').add_to(map_)
map_.add_child(color_map)
svg_style = '<style>svg#legend {font-size: 14px; background-color: white;}</style>'
map_.get_root().header.add_child(folium.Element(svg_style))
# visualising the map
map_

In [ ]:
colormap = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

#Defining the breaks in the colormap 
color_map = cm.LinearColormap(colors = ['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'], vmin = 0, vmax = 1500 )
color_map.caption = 'ppm-m'

# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(methane_plume_tile_2["center"][1], methane_plume_tile_2["center"][0]), zoom_start=13, tiles=None, tooltip = 'test tool tip')
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)


map_layer = TileLayer(
    tiles=methane_plume_tile_2["tiles"][0],
    name='Plume Complex Agriculture',
    overlay='True',
    attr="GHG",
    opacity=1,
)
map_layer.add_to(map_)
folium.LayerControl(collapsed=False, position='bottomleft').add_to(map_)
map_.add_child(color_map)
svg_style = '<style>svg#legend {font-size: 14px; background-color: white;}</style>'
map_.get_root().header.add_child(folium.Element(svg_style))
# visualising the map
map_

In [ ]:
# Creating a dataframe for each plume in the STAC and its corresponding area 

plume_df = pd.DataFrame()
for plume in parse_plume_complexes:
    temp_polygon = geopandas.GeoDataFrame.from_features([plume])['geometry'].values[0]
    geod = Geod(ellps='WGS84')
    ply = wkt.loads(str(temp_polygon))

    temp_dict = {'id':plume['id'], 'geometry':temp_polygon, 'area(km2)':(abs(geod.geometry_area_perimeter(ply)[0])/1e+6), 'date':plume['properties']['datetime'][:10]}
    plume_df = plume_df._append(temp_dict, ignore_index = True)

In [ ]:
plume_df
# plume['properties']['datetime'][:10]

## Visualizing CH4 Emission Plumes in the Permian Basin

In [ ]:
# Converting plume_df into a geo dataframe for further geospatial analysis
geo_temp_df = geopandas.GeoDataFrame(plume_df, geometry=plume_df.geometry, crs = 'EPSG:4326')

In [ ]:
# Selecting all the plume tagets which are under permian basin
# geo_temp_df = geopandas.clip(geo_temp_df, permian_basin)
filtered_geo_temp_df = geo_temp_df[(geo_temp_df['date']<='2023-12-15') & (geo_temp_df['date']>='2023-07-01') ]
filtered_geo_temp_df

colormap = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

color_map = cm.LinearColormap(colors = ['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'], vmin = 0, vmax = 1500 )
color_map.caption = 'ppm-m'
# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.

map_ = folium.Map(location=[32,-102], zoom_start=10, tiles=None)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)

for tile_id in filtered_geo_temp_df['id']: 
    methane_plume_tile = requests.get(
        f"{RASTER_API_URL}/stac/tilejson.json?collection={plume_complexes[tile_id]['collection']}&item={plume_complexes[tile_id]['id']}"
        f"&assets={asset_name}"
        f"&color_formula=gamma+r+1.05&colormap_name={colormap}"
        f"&rescale={rescale_values['min']},{rescale_values['max']}", 
    ).json()
    methane_plume_tile

    map_layer = TileLayer(
        tiles=methane_plume_tile["tiles"][0],
        name=tile_id,
        overlay='True',
        attr="GHG",
        opacity=1,
    )
    map_layer.add_to(map_)

coverage[['fid', 'geometry', 'new_start_time']].explore(
    "fid",
    categorical=False,
    tooltip=[
        "fid",
        "new_start_time",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="Coverage",
    m=map_,
    legend=False,
    show=False
)

map_layer_permian = folium.GeoJson(permian_basin, name= 'Permian Shape')
map_layer_permian.add_to(map_)
folium.LayerControl(collapsed=True, position='bottomleft').add_to(map_)
map_.add_child(color_map)
svg_style = '<style>svg#legend {font-size: 14px; background-color: white;}</style>'
map_.get_root().header.add_child(folium.Element(svg_style))
# visualising the map
map_

In [ ]:
colormap = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

color_map = cm.LinearColormap(colors = ['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'], vmin = 0, vmax = 1500 )
color_map.caption = 'ppm-m'
# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.

map_ = folium.Map(location=[32,-102], zoom_start=10, tiles=None)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)

for tile_id in geo_temp_df['id']: 
    methane_plume_tile = requests.get(
        f"{RASTER_API_URL}/stac/tilejson.json?collection={plume_complexes[tile_id]['collection']}&item={plume_complexes[tile_id]['id']}"
        f"&assets={asset_name}"
        f"&color_formula=gamma+r+1.05&colormap_name={colormap}"
        f"&rescale={rescale_values['min']},{rescale_values['max']}", 
    ).json()
    methane_plume_tile

    map_layer = TileLayer(
        tiles=methane_plume_tile["tiles"][0],
        name=tile_id,
        overlay='True',
        attr="GHG",
        opacity=1,
    )
    map_layer.add_to(map_)

coverage[['fid', 'geometry', 'new_start_time']].explore(
    "fid",
    categorical=False,
    tooltip=[
        "fid",
        "new_start_time",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="Coverage",
    m=map_,
    legend=False,
    show=False
)

map_layer_permian = folium.GeoJson(permian_basin, name= 'Permian Shape')
map_layer_permian.add_to(map_)
folium.LayerControl(collapsed=True, position='bottomleft').add_to(map_)
map_.add_child(color_map)
svg_style = '<style>svg#legend {font-size: 14px; background-color: white;}</style>'
map_.get_root().header.add_child(folium.Element(svg_style))
# visualising the map
map_